In [ ]:
dir_validation_cats = os.path.join(dir_validation, 'cats')
tmp_train = os.path.join(data_path,"train")

In [ ]:
# Configure tensorflow to generate training data from image directory, and normalize the image data
train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(dir_training,
                                                   batch_size = 20,
                                                   class_mode='binary',
                                                   target_size=(48,48))

# Configure tensorflow to generate validation data from image directory, and normalize the image data
validation_datagen = ImageDataGenerator(rescale=1.0/255) 
validation_generator = validation_datagen.flow_from_directory(dir_validation,
                                                             batch_size = 20,
                                                             class_mode='binary',
                                                             target_size=(48,48))

In [ ]:
#Keras model for image classification
model = tf.keras.models.Sequential([
    # Alternating convolution and pooling layers to process the images before analysis
    # The convulation layers accent features from the images,
    # The pooling layers reduce the amount of information to process
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the images for analysis by the nn
    tf.keras.layers.Flatten(),
    # Neural network for image classification 256 neuron hidden layer, with a 1 nueron output layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation ='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(train_generator,
                      epochs=10,
                      steps_per_epoch=1125,
                      validation_steps=125,
                      verbose=1,
                      validation_data=validation_generator)